In [1]:
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential

import cv2

In [18]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'data/dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

test_ds = tf.keras.utils.image_dataset_from_directory(
  'data/test_dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

# def transform_dataset(x, y):
#   # gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
#   print(x.shape)
#   print(y)
#   return x

# train_ds = train_ds.map(transform_dataset)

train_X, train_y = zip(*train_ds)
train_X = np.concatenate(train_X)
train_y = np.concatenate(train_y)


Found 10019 files belonging to 10 classes.
Found 353 files belonging to 10 classes.


In [20]:
train_X.shape, train_y.shape

((10019, 32, 32, 3), (10019, 10))

In [3]:
x, y = next(iter(train_ds))

In [10]:
model = Sequential([
    layers.Input(shape=(None, None, 3)),
    layers.Resizing(32, 32),
    layers.Rescaling(1./255),
    # layers.RandomBrightness(0.2, value_range=(0, 1)),
    layers.RandomFlip('horizontal'),
    # layers.RandomRotation(0.2),
    # layers.RandomZoom(),
    # layers.RandomTranslation(),
    # layers.RandomContrast(),
    # layers.RandomCrop()
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 32, 32, 3)         0         
                                                                 
 rescaling_2 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 random_flip_2 (RandomFlip)  (None, 32, 32, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                      

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model.fit(train_X, train_y, validation_data=test_ds, epochs=40, batch_size=64)

Epoch 1/40
157/157 [==============================] - 5s 27ms/step - loss: 0.0118 - accuracy: 0.9965 - val_loss: 8.9760 - val_accuracy: 0.3881
Epoch 2/40
157/157 [==============================] - 5s 31ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 7.4705 - val_accuracy: 0.3909
Epoch 3/40
157/157 [==============================] - 5s 32ms/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 7.7017 - val_accuracy: 0.4108
Epoch 4/40
157/157 [==============================] - 5s 31ms/step - loss: 0.0024 - accuracy: 0.9992 - val_loss: 8.1972 - val_accuracy: 0.3938
Epoch 5/40
157/157 [==============================] - 5s 33ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 9.6441 - val_accuracy: 0.3938
Epoch 6/40
157/157 [==============================] - 5s 31ms/step - loss: 8.3615e-04 - accuracy: 0.9998 - val_loss: 8.5885 - val_accuracy: 0.3909
Epoch 7/40
157/157 [==============================] - 5s 33ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 9.4853 - val_accuracy: 0.

In [22]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

In [203]:
X.shape, y.shape

((72, 32, 32, 3), (72, 10))

In [23]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

predictions = model.predict(X)
report = classification_report(np.argmax(y, axis=1), np.argmax(predictions, axis=1))
print(report)

12/12 [==============================] - 0s 26ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.55      0.66      0.60        50
           4       0.84      0.54      0.66        50
           5       0.35      0.74      0.47        50
           6       0.30      0.14      0.19        50
           7       1.00      0.04      0.08        50
           8       0.36      0.10      0.16        50
           9       0.36      0.60      0.45        50

    accuracy                           0.40       353
   macro avg       0.38      0.28      0.26       353
weighted avg       0.53      0.40      0.37       353

